# EDA - UAT 403 (București)

### Data Sources
Three aggregated datasets containing daily measurements:
1. **Cloud Cover** (`Aggregated_CloudCover/UAT_403.csv`)
2. **Humidity and Wind** (`Aggregated_Humidity_and_Wind/UAT_403.csv`)
3. **Temperature and UTCI** (`Aggregated_Temp_and_UTCI/UAT_403.csv`)

### Objectives
1. Load and merge all datasets, validate date alignment
2. Document all variables with narrative descriptions
3. Compute descriptive statistics (mean, std, min, max, quartiles)
4. Visualize time series evolution for each variable
5. Analyze correlations between variables
6. Identify anomalies/peaks in the data

---
## 1. Setup and Data Loading

In [ ]:
%pip install -r requirements.txt

In [ ]:
# Install nbstripout to remove output from Jupyter notebooks before committing to version control
import subprocess
import os
from pathlib import Path
import sys

# Install nbstripout if not already installed
subprocess.run([sys.executable, "-m", "pip", "install", "nbstripout"], capture_output=True)

# Find repo root
current = Path(os.getcwd())
repo_root = None
for parent in [current] + list(current.parents):
    if (parent / ".git").exists():
        repo_root = str(parent)
        break

if repo_root is None:
    print("Could not find git repo root!")
else:
    # Find git on Windows (not needed on Mac/Linux as git is usually on PATH)
    env = os.environ.copy()
    if sys.platform == "win32":
        common_paths = [
            r"C:\Program Files\Git\bin",
            r"C:\Program Files (x86)\Git\bin",
            os.path.expanduser(r"~\AppData\Local\Programs\Git\bin"),
        ]
        for p in common_paths:
            if Path(p, "git.exe").exists():
                env["PATH"] = p + ";" + env["PATH"]
                break

    result = subprocess.run(
        "nbstripout --install",
        cwd=repo_root, shell=True,
        capture_output=True, text=True, env=env
    )

    if result.returncode == 0:
        print("nbstripout installed — notebook outputs will be stripped before commits")
    else:
        print(f"Installation failed: {result.stderr}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
sns.set_palette('husl')

DATA_DIR = Path('../data/unziped_data')
UAT_ID = '403'

In [ ]:
df_cloud = pd.read_csv(DATA_DIR / 'Aggregated_CloudCover' / f'UAT_{UAT_ID}.csv')
df_humidity = pd.read_csv(DATA_DIR / 'Aggregated_Humidity_and_Wind' / f'UAT_{UAT_ID}.csv')
df_temp = pd.read_csv(DATA_DIR / 'Aggregated_Temp_and_UTCI' / f'UAT_{UAT_ID}.csv')

print(f'Cloud Cover dataset: {df_cloud.shape[0]} rows, {df_cloud.shape[1]} columns')
print(f'Humidity and Wind dataset: {df_humidity.shape[0]} rows, {df_humidity.shape[1]} columns')
print(f'Temperature and UTCI dataset: {df_temp.shape[0]} rows, {df_temp.shape[1]} columns')

---
## 2. Data Alignment Validation

We need to verify that all three datasets are aligned by date.

In [ ]:
df_cloud['date'] = pd.to_datetime(df_cloud['date'])
df_humidity['date'] = pd.to_datetime(df_humidity['date'])
df_temp['date'] = pd.to_datetime(df_temp['date'])

print(f'Cloud Cover:      {df_cloud["date"].min()} to {df_cloud["date"].max()}')
print(f'Humidity & Wind:  {df_humidity["date"].min()} to {df_humidity["date"].max()}')
print(f'Temperature:      {df_temp["date"].min()} to {df_temp["date"].max()}')

In [ ]:
dates_cloud = set(df_cloud['date'])
dates_humidity = set(df_humidity['date'])
dates_temp = set(df_temp['date'])

all_dates = dates_cloud | dates_humidity | dates_temp
common_dates = dates_cloud & dates_humidity & dates_temp

print(f'Total unique dates across all datasets: {len(all_dates)}')
print(f'Dates present in ALL datasets: {len(common_dates)}')

only_in_cloud = dates_cloud - common_dates
only_in_humidity = dates_humidity - common_dates
only_in_temp = dates_temp - common_dates

if only_in_cloud:
    print(f'Discrepancy: {len(only_in_cloud)} dates ONLY in Cloud Cover dataset')
if only_in_humidity:
    print(f'Discrepancy: {len(only_in_humidity)} dates ONLY in Humidity & Wind dataset')
if only_in_temp:
    print(f'Discrepancy: {len(only_in_temp)} dates ONLY in Temperature dataset')

if len(common_dates) != len(all_dates):
    print(f'Warning: Datasets have {len(all_dates) - len(common_dates)} misaligned dates!')

In [ ]:
def check_date_continuity(dates, name):
    dates_sorted = sorted(dates)
    expected_dates = pd.date_range(start=dates_sorted[0], end=dates_sorted[-1], freq='D')
    missing = set(expected_dates) - set(dates_sorted)
    if missing:
        print(f'{name}: {len(missing)} missing dates (gaps)')
    else:
        print(f'{name}: Continuous daily data (no gaps)')

check_date_continuity(dates_cloud, 'Cloud Cover')
check_date_continuity(dates_humidity, 'Humidity & Wind')
check_date_continuity(dates_temp, 'Temperature')

In [ ]:
df_merged = pd.merge(
    df_cloud[['date', 'CC']], 
    df_humidity[['date', 'RH_AVG', 'RH_MN', 'RH_MX', 'WS_AVG', 'WS_MX']], 
    on='date', how='inner'
)
df = pd.merge(
    df_merged, 
    df_temp[['date', 'TG', 'TN', 'TX', 'UTCI_MN', 'UTCI_MX']], 
    on='date', how='inner'
)
df = df.sort_values('date').reset_index(drop=True)

print(f'Merged dataset: {df.shape[0]} rows, {df.shape[1]} columns')
print(f'Date range: {df["date"].min()} to {df["date"].max()}')
print(f'Duration: ~{(df["date"].max() - df["date"].min()).days / 365:.1f} years')

In [ ]:
df.head(10)

---
## 3. Variable Documentation

| Variable | Name | Description | Unit | Domain |
|----------|------|-------------|------|--------|
| CC | Cloud Cover | Daily average fraction of sky covered | 0-1 | 0=clear, 1=overcast |
| RH_AVG | Relative Humidity (Avg) | Daily average humidity | % | 0-100 |
| RH_MN | Relative Humidity (Min) | Daily minimum humidity | % | 0-100 |
| RH_MX | Relative Humidity (Max) | Daily maximum humidity | % | 0-100 |
| WS_AVG | Wind Speed (Avg) | Daily average wind speed | m/s | >= 0 |
| WS_MX | Wind Speed (Max) | Daily max wind speed | m/s | >= 0 |
| TG | Temperature (Mean) | Daily mean temperature | C | -30 to +45 |
| TN | Temperature (Min) | Daily min temperature | C | -30 to +40 |
| TX | Temperature (Max) | Daily max temperature | C | -25 to +45 |
| UTCI_MN | UTCI (Min) | Thermal comfort index min | C | -40 to +46 |
| UTCI_MX | UTCI (Max) | Thermal comfort index max | C | -40 to +46 |

---
## 4. Descriptive Statistics

In [ ]:
numeric_cols = ['CC', 'RH_AVG', 'RH_MN', 'RH_MX', 'WS_AVG', 'WS_MX', 'TG', 'TN', 'TX', 'UTCI_MN', 'UTCI_MX']

stats = df[numeric_cols].describe().T
stats['median'] = df[numeric_cols].median()
stats['variance'] = df[numeric_cols].var()
stats['skewness'] = df[numeric_cols].skew()
stats['kurtosis'] = df[numeric_cols].kurtosis()
stats.round(3)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

skew_vals = df[numeric_cols].skew()
colors_skew = ['green' if abs(v) < 0.5 else 'orange' if abs(v) < 1 else 'red' for v in skew_vals]
axes[0].barh(numeric_cols, skew_vals, color=colors_skew, edgecolor='black')
axes[0].axvline(0, color='black', linewidth=1)
axes[0].axvline(-0.5, color='gray', linestyle='--', alpha=0.5)
axes[0].axvline(0.5, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Skewness')
axes[0].set_title('Skewness by variable\n(Green: symmetric, Orange: moderate, Red: high)', fontweight='bold')
for i, v in enumerate(skew_vals):
    axes[0].text(v + 0.05, i, f'{v:.2f}', va='center', fontsize=9)

kurt_vals = df[numeric_cols].kurtosis()
colors_kurt = ['green' if abs(v) < 1 else 'orange' if abs(v) < 3 else 'red' for v in kurt_vals]
axes[1].barh(numeric_cols, kurt_vals, color=colors_kurt, edgecolor='black')
axes[1].axvline(0, color='black', linewidth=1)
axes[1].axvline(-1, color='gray', linestyle='--', alpha=0.5)
axes[1].axvline(1, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Kurtosis')
axes[1].set_title('Kurtosis by variable\n(Green: normal tails, Orange: moderate, Red: heavy tails)', fontweight='bold')
for i, v in enumerate(kurt_vals):
    axes[1].text(v + 0.05, i, f'{v:.2f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
missing = df.isnull().sum()
if missing.sum() == 0:
    print('No missing values!')
else:
    print(missing[missing > 0])

In [ ]:
print(f'TN <= TG <= TX: {((df["TN"] <= df["TG"]) & (df["TG"] <= df["TX"])).all()}')
print(f'RH_MN <= RH_AVG <= RH_MX: {((df["RH_MN"] <= df["RH_AVG"]) & (df["RH_AVG"] <= df["RH_MX"])).all()}')
print(f'WS_AVG <= WS_MX: {(df["WS_AVG"] <= df["WS_MX"]).all()}')
print(f'UTCI_MN <= UTCI_MX: {(df["UTCI_MN"] <= df["UTCI_MX"]).all()}')

---
## 5. Distribution Analysis

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(16, 14))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    ax.hist(df[col], bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: {df[col].mean():.2f}')
    ax.axvline(df[col].median(), color='orange', linestyle=':', label=f'Median: {df[col].median():.2f}')
    ax.set_title(f'{col} Distribution', fontweight='bold')
    ax.legend(fontsize=8)

axes[-1].set_visible(False)
plt.suptitle('Distribution of All Variables', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 6. Time Series Visualization

In [ ]:
df_ts = df.set_index('date')

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.fill_between(df_ts.index, df_ts['TN'], df_ts['TX'], alpha=0.3, color='red', label='TN-TX Range')
ax.plot(df_ts.index, df_ts['TG'], linewidth=0.5, color='darkred', label='TG (Mean)')
ax.axhline(0, color='blue', linestyle='--', alpha=0.5, label='0C')
ax.set_title('Temperature - Daily Time Series', fontweight='bold')
ax.set_ylabel('Temperature (C)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))
ax.fill_between(df_ts.index, 0, df_ts['WS_MX'], alpha=0.3, color='orange', label='WS_MX')
ax.plot(df_ts.index, df_ts['WS_AVG'], linewidth=0.5, color='darkorange', label='WS_AVG')
ax.set_title('Wind Speed - Daily Time Series', fontweight='bold')
ax.set_ylabel('Wind Speed (m/s)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.fill_between(df_ts.index, df_ts['UTCI_MN'], df_ts['UTCI_MX'], alpha=0.3, color='purple', label='UTCI Range')
ax.axhspan(9, 26, alpha=0.1, color='green', label='Comfortable (9-26C)')
ax.set_title('UTCI - Daily Time Series', fontweight='bold')
ax.set_ylabel('UTCI (C)')
ax.legend()
plt.tight_layout()
plt.show()

---
## 7. Seasonal Patterns

In [ ]:
df_ts['month'] = df_ts.index.month
df_ts['year'] = df_ts.index.year

monthly_avg = df_ts.groupby('month')[numeric_cols].mean()
monthly_std = df_ts.groupby('month')[numeric_cols].std()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0,0].bar(monthly_avg.index, monthly_avg['CC'], color='steelblue')
axes[0,0].set_title('Cloud Cover by Month')
axes[0,0].set_xticks(range(1,13))

axes[0,1].bar(monthly_avg.index, monthly_avg['RH_AVG'], color='green')
axes[0,1].set_title('Humidity by Month')
axes[0,1].set_xticks(range(1,13))

axes[1,0].errorbar(monthly_avg.index, monthly_avg['TG'], yerr=monthly_std['TG'],
                   fmt='o-', color='darkred', ecolor='black', capsize=5, capthick=1.5,
                   elinewidth=1.5, label='TG Mean +- 1σ')
axes[1,0].fill_between(monthly_avg.index, monthly_avg['TN'], monthly_avg['TX'], alpha=0.2, color='red', label='TN-TX Range')
axes[1,0].set_title('Temperature by Month')
axes[1,0].set_xticks(range(1,13))
axes[1,0].legend(fontsize=8)

utci_mid_monthly = (monthly_avg['UTCI_MN'] + monthly_avg['UTCI_MX']) / 2
utci_mid_std = df_ts.assign(UTCI_MID=(df_ts['UTCI_MN']+df_ts['UTCI_MX'])/2).groupby('month')['UTCI_MID'].std()
axes[1,1].errorbar(monthly_avg.index, utci_mid_monthly, yerr=utci_mid_std,
                   fmt='o-', color='purple', ecolor='black', capsize=5, capthick=1.5,
                   elinewidth=1.5, label='UTCI Mid +- 1σ')
axes[1,1].fill_between(monthly_avg.index, monthly_avg['UTCI_MN'], monthly_avg['UTCI_MX'], alpha=0.2, color='purple', label='UTCI Range')
axes[1,1].set_title('UTCI by Month')
axes[1,1].set_xticks(range(1,13))
axes[1,1].legend(fontsize=8)

plt.suptitle('Seasonal Patterns', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
yearly_avg = df_ts.groupby('year')[numeric_cols].mean()
yearly_std = df_ts.groupby('year')[numeric_cols].std()
yearly_count = df_ts.groupby('year')[numeric_cols].count()
yearly_sem = yearly_std['TG'] / np.sqrt(yearly_count['TG'])

fig, ax = plt.subplots(figsize=(14, 5))
ax.errorbar(yearly_avg.index, yearly_avg['TG'], yerr=yearly_sem,
            fmt='o', color='darkred', ecolor='black', capsize=4, capthick=1.2,
            elinewidth=1.2, markersize=5, zorder=3, label='TG Yearly Mean +- SEM')
ax.plot(yearly_avg.index, yearly_avg['TG'], '-', color='darkred', linewidth=0.8, alpha=0.5, zorder=2)
z = np.polyfit(yearly_avg.index, yearly_avg['TG'], 1)
p = np.poly1d(z)
ax.plot(yearly_avg.index, p(yearly_avg.index), '--', color='blue', linewidth=2, zorder=4,
        label=f'Trend: {z[0]*10:.3f}°C/decade')
ax.set_title('Mean Temperature - Yearly Trend', fontweight='bold')
ax.set_ylabel('Temperature (°C)')
ax.set_xlabel('Year')
ax.legend()
plt.tight_layout()
plt.show()

---
## 8. Correlation Analysis

In [ ]:
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, square=True)
plt.title('Correlation Heatmap', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print('Strong correlations (|r| > 0.7):')
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        r = corr_matrix.iloc[i, j]
        if abs(r) > 0.7:
            print(f'{corr_matrix.columns[i]} <-> {corr_matrix.columns[j]}: r = {r:.3f}')

---
## 9. Anomaly Detection

In [ ]:
print(f'Hottest day: {df.loc[df["TX"].idxmax(), "date"]} - TX: {df["TX"].max():.1f}C')
print(f'Coldest day: {df.loc[df["TN"].idxmin(), "date"]} - TN: {df["TN"].min():.1f}C')
print(f'Windiest day: {df.loc[df["WS_MX"].idxmax(), "date"]} - WS_MX: {df["WS_MX"].max():.1f} m/s')
print(f'Max heat stress: {df.loc[df["UTCI_MX"].idxmax(), "date"]} - UTCI_MX: {df["UTCI_MX"].max():.1f}C')
print(f'Max cold stress: {df.loc[df["UTCI_MN"].idxmin(), "date"]} - UTCI_MN: {df["UTCI_MN"].min():.1f}C')

In [ ]:
MORTALITY_RISK_THRESHOLD = 38
extreme_days = df[(df['UTCI_MX'] > MORTALITY_RISK_THRESHOLD) | (df['TX'] > MORTALITY_RISK_THRESHOLD)].copy()
extreme_days = extreme_days.sort_values('UTCI_MX', ascending=False)

print('Top 10 most extreme days - Highest Mortality Risk\n')
print(extreme_days[['date', 'TX', 'TN', 'UTCI_MX', 'RH_MX', 'RH_AVG']].head(10).to_string(index=False))

---
## 10. Heat Wave Analysis (Mortality Risk)

The heatwaves are computed with the below factors:
- **UTCI_MX > 32°C** (strong heat stress threshold)
- **TX > 35°C** (extreme temperature)
- Combined with humidity conditions (RH_MX)

In [ ]:
UTCI_REFERENCE = 'https://climate-adapt.eea.europa.eu/en/metadata/indicators/high-utci-days'

UTCI_HEAT_STRESS = 32
TX_EXTREME = 35
HUMIDITY_HUMID_HEAT = 80  # Humid heat (sweating ineffective)
HUMIDITY_DRY_HEAT = 50 # Dry heat (dehydration risk)

df['heat_stress_day'] = (df['UTCI_MX'] > UTCI_HEAT_STRESS) | (df['TX'] > TX_EXTREME)
df['humid_heat'] = df['heat_stress_day'] & (df['RH_MX'] > HUMIDITY_HUMID_HEAT)
df['dry_heat'] = df['heat_stress_day'] & (df['RH_MX'] < HUMIDITY_DRY_HEAT)

print(f'Heat stress days (UTCI_MX > {UTCI_HEAT_STRESS} or TX > {TX_EXTREME}): {df["heat_stress_day"].sum()} ({df["heat_stress_day"].mean()*100:.1f}%)')
print(f'  - Humid heat days (RH_MX > {HUMIDITY_HUMID_HEAT}%): {df["humid_heat"].sum()} ({df["humid_heat"].mean()*100:.2f}%)')
print(f'  - Dry heat days (RH_MX < {HUMIDITY_DRY_HEAT}%): {df["dry_heat"].sum()} ({df["dry_heat"].mean()*100:.2f}%)')

In [ ]:
heat_by_year = df.groupby(df['date'].dt.year).agg({
    'heat_stress_day': 'sum',
    'humid_heat': 'sum',
    'dry_heat': 'sum'
})

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(heat_by_year.index, heat_by_year['heat_stress_day'], color='red', alpha=0.7, label='Total Heat Stress Days')
ax.bar(heat_by_year.index, heat_by_year['humid_heat'], color='darkred', alpha=0.9, label='Humid Heat RH_MX > ' + str(HUMIDITY_HUMID_HEAT) + '%')
ax.bar(heat_by_year.index, heat_by_year['dry_heat'], color='darkgreen', alpha=1.0, label='Dry Heat RH_MX < ' + str(HUMIDITY_DRY_HEAT) + '%')

z = np.polyfit(heat_by_year.index, heat_by_year['heat_stress_day'], 1)
p = np.poly1d(z)
ax.plot(heat_by_year.index, p(heat_by_year.index), '--', color='black', linewidth=2,
        label=f'Trend: {z[0]:.2f} days/year')

ax.set_xlabel('Year')
ax.set_ylabel('Number of Days')
ax.set_title('Heat Stress Days per Year', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

print(f'\nTrend: {z[0]:.2f} additional heat stress days per year')
print(f'       ({z[0]*10:.1f} more days per decade)')

In [ ]:
extreme_days = df[(df['UTCI_MX'] > MORTALITY_RISK_THRESHOLD) | (df['TX'] > MORTALITY_RISK_THRESHOLD)].copy()
extreme_days = extreme_days.sort_values('UTCI_MX', ascending=False)

TOP_N_DAYS = 10
print(f'Top {TOP_N_DAYS} most extreme days (UTCI_MX > {MORTALITY_RISK_THRESHOLD}C or TX > {MORTALITY_RISK_THRESHOLD}C):\n')
print(extreme_days[['date', 'TX', 'TN', 'UTCI_MX', 'RH_MX', 'RH_AVG']].head(TOP_N_DAYS).to_string(index=False))

In [ ]:
heat_by_month = df[df['heat_stress_day']].groupby(df[df['heat_stress_day']]['date'].dt.month).size()

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(heat_by_month.index, heat_by_month.values, color='red', edgecolor='black')
ax.set_xlabel('Month')
ax.set_ylabel('Total Heat Stress Days for all years')
ax.set_title('Monthly Distribution of heat stress', fontweight='bold')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

for bar, val in zip(bars, heat_by_month.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, str(val), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 11. Summary

In [ ]:
print(f'Date range: {df["date"].min()} to {df["date"].max()}')
print(f'Total records: {len(df):,}')
print(f'Mean Temperature: {df["TG"].mean():.2f}C (std: {df["TG"].std():.2f})')
print(f'Temperature range: {df["TN"].min():.1f}C to {df["TX"].max():.1f}C')
print(f'Mean Cloud Cover: {df["CC"].mean():.2f}')
print(f'Mean Humidity: {df["RH_AVG"].mean():.1f}%')
print(f'Mean Wind Speed: {df["WS_AVG"].mean():.2f} m/s')

In [ ]:
df.to_csv('../data/processed_data/processed_UAT_' + UAT_ID +'.csv', index=False)